In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import requests
import numpy as np

In [2]:
def fetch_upcoming_fixtures(api_url):
    response = requests.get(api_url)
    
    if response.status_code == 200:
        fixtures_data = response.json().get('fixtures', [])
        if fixtures_data:
            upcoming_fixtures = pd.DataFrame({
                'Home_Team_Name': [fixture['home_team'] for fixture in fixtures_data],
                'Away_Team_Name': [fixture['away_team'] for fixture in fixtures_data],
            })
            return upcoming_fixtures
        else:
            print('No upcoming fixtures found.')
    else:
        print(f'Failed to fetch upcoming fixtures. Status code: {response.status_code}')

In [3]:
api_url = 'http://localhost:5000/api/fixtures'
upcoming_fixtures = fetch_upcoming_fixtures(api_url)
print(upcoming_fixtures)

      Home_Team_Name     Away_Team_Name
0          Brentford            Arsenal
1            Burnley           West Ham
2            Everton  Manchester United
3             Fulham             Wolves
4              Luton     Crystal Palace
..               ...                ...
255   Crystal Palace        Aston Villa
256        Liverpool             Wolves
257            Luton             Fulham
258  Manchester City           West Ham
259    Sheffield Utd          Tottenham

[260 rows x 2 columns]


In [4]:
df = pd.read_csv('../data/Premier_League_all_match_stats_2015-2023.csv')

In [5]:
label_encoder = LabelEncoder()
df['Home_Team_Name'] = label_encoder.fit_transform(df['Home_Team_Name'])
df['Away_Team_Name'] = label_encoder.transform(df['Away_Team_Name'])

upcoming_fixtures['Home_Team_Name'] = label_encoder.transform(upcoming_fixtures['Home_Team_Name'])
upcoming_fixtures['Away_Team_Name'] = label_encoder.transform(upcoming_fixtures['Away_Team_Name'])

df['Home_passes_%'] = df['Home_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Away_passes_%'] = df['Away_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Home_ball_possession'] = df['Home_ball_possession'].str.rstrip('%').astype('float') / 100.0
df['Away_ball_possession'] = df['Away_ball_possession'].str.rstrip('%').astype('float') / 100.0

df = df.fillna(0)

df = df.drop(['Home_expected_goals', 'Away_expected_goals', 'Away_blocked_shots', 'Away_goalkeeper_saves', 'Away_offsides', 'Away_passes_%', 'Season', 'Match_Date', 'Away_passes_accurate', 'Away_shots_insidebox', 'Away_shots_off_goal', 'Away_shots_on_goal', 'Away_shots_outsidebox', 'Home_blocked_shots', 'Home_goalkeeper_saves', 'Home_offsides', 'Home_passes_%', 'Home_passes_accurate', 'Away_total_passes', 'Home_shots_insidebox', 'Home_shots_off_goal', 'Home_shots_on_goal', 'Home_shots_outsidebox', 'Home_total_passes'], axis=1)

In [6]:
target_columns = ['Home_Goals', 'Away_Goals',
       'Home_total_shots', 'Home_corner_kicks', 'Home_ball_possession',
       'Home_yellow_cards', 'Away_total_shots', 'Away_corner_kicks',
       'Away_ball_possession', 'Away_yellow_cards', 'Away_red_cards',
       'Home_red_cards', 'Home_fouls', 'Away_fouls']

In [7]:
models = {}

for target in target_columns:
    features = df.loc[:, df.columns != target]
    X = features
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    models[target] = model

    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f'Mean Squared Error for {target}: {mse}')

Mean Squared Error for Home_Goals: 1.399967652379675
Mean Squared Error for Away_Goals: 1.052233864229563
Mean Squared Error for Home_total_shots: 15.576261110842918
Mean Squared Error for Home_corner_kicks: 5.939912187364706
Mean Squared Error for Home_ball_possession: 5.435507432457149e-31
Mean Squared Error for Home_yellow_cards: 1.2238640822098719
Mean Squared Error for Away_total_shots: 13.524499708865296
Mean Squared Error for Away_corner_kicks: 5.111503099206632
Mean Squared Error for Away_ball_possession: 5.522727401703309e-31
Mean Squared Error for Away_yellow_cards: 1.4760061419420503
Mean Squared Error for Away_red_cards: 0.0461957373884766
Mean Squared Error for Home_red_cards: 0.06530939661954223
Mean Squared Error for Home_fouls: 13.046643262712568
Mean Squared Error for Away_fouls: 13.592753924413604


In [8]:
for index, row in upcoming_fixtures.iterrows():
    home_team = label_encoder.inverse_transform([row['Home_Team_Name']])[0]
    away_team = label_encoder.inverse_transform([row['Away_Team_Name']])[0]
    teams_involved = f"{home_team} vs {away_team}"
    print(f"\nTeams: {teams_involved}")

    for target in target_columns:
        row_copy = row.copy()
        
        for col, val in zip(target_columns, [0] * len(target_columns)):
            if col != target:
                row_copy[col] = val

        model = models[target]

        reshaped_row = row_copy.values.reshape(1, -1)
        y_pred = model.predict(reshaped_row)

        print(f"{target} Prediction: {y_pred[0]}")
        
    print("\n")


Teams: Brentford vs Arsenal
Home_Goals Prediction: 1.2164532532913042
Away_Goals Prediction: 0.679202778523987
Home_total_shots Prediction: 10.816079830770411
Home_corner_kicks Prediction: 3.742019889782133
Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.7981062373310494
Away_total_shots Prediction: 8.278065001631877
Away_corner_kicks Prediction: 3.381786317362552
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.0584498705565315
Away_red_cards Prediction: 0.004067298087542007
Home_red_cards Prediction: -0.03473991402271359
Home_fouls Prediction: 3.6546493294568267
Away_fouls Prediction: 4.773144190486874



Teams: Burnley vs West Ham
Home_Goals Prediction: 1.2599448580641954
Away_Goals Prediction: 0.6460282016977738
Home_total_shots Prediction: 10.45931875683089
Home_corner_kicks Prediction: 3.739920196780487
Home_ball_possession Prediction: 1.0000000000000027
Home_yellow_cards Prediction: 0.7769462622358888
Away_t

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali


Home_total_shots Prediction: 10.628347101958353
Home_corner_kicks Prediction: 3.699746382208938
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7800933811521916
Away_total_shots Prediction: 8.36086012904287
Away_corner_kicks Prediction: 3.364214276232202
Away_ball_possession Prediction: 0.9999999999999974
Away_yellow_cards Prediction: 1.089555302858914
Away_red_cards Prediction: 0.0003014847940021777
Home_red_cards Prediction: -0.03631688615039512
Home_fouls Prediction: 3.273828753900134
Away_fouls Prediction: 4.5561296505979785



Teams: Fulham vs Wolves
Home_Goals Prediction: 1.2559937634611862
Away_Goals Prediction: 0.6440583415815225
Home_total_shots Prediction: 10.467617574848722
Home_corner_kicks Prediction: 3.6979770471367504
Home_ball_possession Prediction: 1.0000000000000029
Home_yellow_cards Prediction: 0.7704227349867437
Away_total_shots Prediction: 8.414565686918817
Away_corner_kicks Prediction: 3.385637013063991
Away_ball_possession Pre

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Home_corner_kicks Prediction: 3.6011399535647026
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7737438253668383
Away_total_shots Prediction: 8.353025716066552
Away_corner_kicks Prediction: 3.2345932260791734
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.129366196634633
Away_red_cards Prediction: 0.003012240428069248
Home_red_cards Prediction: -0.02956630374477634
Home_fouls Prediction: 3.7603928668978126
Away_fouls Prediction: 3.8594951072408517



Teams: Nottingham Forest vs Brighton
Home_Goals Prediction: 1.201733682313687
Away_Goals Prediction: 0.6715056327438026
Home_total_shots Prediction: 10.845261871256827
Home_corner_kicks Prediction: 3.5827318529454466
Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.7731955882666806
Away_total_shots Prediction: 8.347973425527758
Away_corner_kicks Prediction: 3.208811997837394
Away_ball_possession Prediction: 0.9999999999999977
Away_y

...

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Away_red_cards Prediction: -0.00030304130892318606
Home_red_cards Prediction: -0.03912250401045673
Home_fouls Prediction: 3.116133343918213
Away_fouls Prediction: 4.9089508057417905



Teams: Brighton vs Manchester United
Home_Goals Prediction: 1.24188141210047
Away_Goals Prediction: 0.6599695233805117
Home_total_shots Prediction: 10.608280944033766
Home_corner_kicks Prediction: 3.742169190900252
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7860106808132483
Away_total_shots Prediction: 8.34628196677114
Away_corner_kicks Prediction: 3.412064112504709
Away_ball_possession Prediction: 0.9999999999999974
Away_yellow_cards Prediction: 1.0695031088572255
Away_red_cards Prediction: 0.00016190799126980732
Home_red_cards Prediction: -0.03828968520146787
Home_fouls Prediction: 3.187292244376751
Away_fouls Prediction: 4.838830061946579



Teams: Burnley vs Nottingham Forest
Home_Goals Prediction: 1.2466883749424027
Away_Goals Prediction: 0.6555536819271011
H

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Away_total_shots Prediction: 8.296271015719434
Away_corner_kicks Prediction: 3.3361502456676373
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.0788933311971285
Away_red_cards Prediction: 0.003988357920451118
Home_red_cards Prediction: -0.032986244471028
Home_fouls Prediction: 3.714260039703279
Away_fouls Prediction: 4.497234109901019



Teams: Liverpool vs Wolves
Home_Goals Prediction: 1.250569726289089
Away_Goals Prediction: 0.6431468681574187
Home_total_shots Prediction: 10.48768373277331
Home_corner_kicks Prediction: 3.6555542384454363
Home_ball_possession Prediction: 1.0000000000000029
Home_yellow_cards Prediction: 0.764505435325687
Away_total_shots Prediction: 8.429143849190545
Away_corner_kicks Prediction: 3.337787176791484
Away_ball_possession Prediction: 0.9999999999999974
Away_yellow_cards Prediction: 1.1190956686056617
Away_red_cards Prediction: -0.002590260620244614
Home_red_cards Prediction: -0.03701734028052801
Home_fouls Prediction: 3.

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali